In [165]:
from bs4 import BeautifulSoup
import requests
import time
import json

import pandas as pd

In [166]:
# Función que obtiene el enlace de cada equipo
def obtenerEnlacesEquipos():    
    url_principal = 'https://www.laliga.com/laliga-easports/clubes'
    peticion = requests.get(url_principal)  # Petición a la página web 
    time.sleep(2)
    sopa = BeautifulSoup(peticion.text, 'lxml') # Todo el contenido de la página
    contenido = sopa.find("div", class_="styled__GridStyled-sc-skzs8h-0 fVbFTE")   
    elementos = contenido.find_all("a", class_="link")  # Buscar en la página los elementos donde están los enlaces
    enlaces_equipos = []
    for i in range(len(elementos)):
        enlace = elementos[i].get('href')   # Obtener los enlaces del atributo href
        enlaces_equipos.append(enlace)
    return enlaces_equipos

In [167]:
# Función que obtiene el enlace de cada jugador
def obtenerEnlacesJugadores():
    enlaces_jugadores = []
    enlaces_equipos = obtenerEnlacesEquipos()
    for i in range(len(enlaces_equipos)):
        url_equipo = 'https://www.laliga.com' + enlaces_equipos[i] + '/plantilla'
        peticion = requests.get(url_equipo)
        time.sleep(2)
        sopa = BeautifulSoup(peticion.text, 'lxml')
        contenido = sopa.find("div", class_="styled__SquadListContainer-sc-sx1q1t-0 hwjXzG")
        elementos = contenido.find_all("a", class_="link")
        for i in range(len(elementos)):
            enlace_jugador = elementos[i].get('href')
            enlaces_jugadores.append(enlace_jugador)
    return enlaces_jugadores

In [168]:
lista_jugadores = obtenerEnlacesJugadores()
lista_jugadores

['/jugador/julen-agirrezabala',
 '/jugador/unai-simon',
 '/jugador/vivian',
 '/jugador/i-lekue',
 '/jugador/imanol-garcia-de-albeniz',
 '/jugador/paredes-2',
 '/jugador/yeray-2',
 '/jugador/o-sancet-1',
 '/jugador/r-de-galarreta',
 '/jugador/benat-prados',
 '/jugador/vencedor',
 '/jugador/guruzeta',
 '/jugador/williams-jr',
 '/jugador/malcom-adu-ares',
 '/jugador/villalibre',
 '/jugador/williams',
 '/jugador/javier-marton',
 '/jugador/nicolas-serrano',
 '/jugador/horatiu-moldovan',
 '/jugador/oblak',
 '/jugador/reinildo',
 '/jugador/nahuel-molina',
 '/jugador/j-m-gimenez',
 '/jugador/santiago-mourino',
 '/jugador/de-paul',
 '/jugador/saul-1',
 '/jugador/arthur-vermeeren',
 '/jugador/pablo-barrios-rivas',
 '/jugador/riquelme',
 '/jugador/koke',
 '/jugador/m-llorente',
 '/jugador/lemar',
 '/jugador/javi-galan',
 '/jugador/morata',
 '/jugador/samuel-lino',
 '/jugador/griezmann',
 '/jugador/correa',
 '/jugador/aitor-fdez',
 '/jugador/s-herrera',
 '/jugador/juan-cruz-1',
 '/jugador/areso',


In [194]:
def obtenerDatosJugadores():    # Función para obtener los datos de todos los jugadores
    nombres = []
    apodos = []
    fechas_nacimiento = []
    pesos = []
    alturas = []
    paises = []
    posiciones = []
    equipos = []
    dorsales = []
    fotos = []
    for i in range(len(lista_jugadores)):
        url_jugador = 'https://www.laliga.com' + lista_jugadores[i]
        peticion = requests.get(url_jugador)    # Petición a la página web
        time.sleep(2)
        sopa = BeautifulSoup(peticion.text, 'lxml') # Contenido de la web
        script_jugador = sopa.find_all('script', id='__NEXT_DATA__')
        script_jugador = script_jugador[0].string   # Script con los datos del jugador
        dicc_jugador = json.loads(script_jugador)   # Pasar la cadena de caracteres a un formato más legible
        dicc_jugador = dicc_jugador['props']['pageProps']['player'] # Obtener el diccionario del jugador
        lista_claves = list(dicc_jugador.keys())    # Lista con las claves del diccionario
        posicion = dicc_jugador['squad']['position']['name']    
        if posicion == 'Entrenador': # En principio no nos interesa la informacion de los entrenadores
            pass
        else:
            if 'name' in lista_claves:
                nombre = dicc_jugador['name']
            else: 
                nombre = ''
            if 'nickname' in lista_claves:
                apodo = dicc_jugador['nickname']
            else:
                apodo = ''
            if 'date_of_birth' in lista_claves:
                fecha_nacimiento = dicc_jugador['date_of_birth']
            else: 
                fecha_nacimiento = ''
            if 'weight' in lista_claves:
                peso = dicc_jugador['weight']
            else:
                peso = 0
            if 'height' in lista_claves:
                altura = dicc_jugador['height']
            else: 
                altura = 0
            equipo = dicc_jugador['team']['name']   
            pais = dicc_jugador['country']['id']
            try:
                dorsal = dicc_jugador['squad']['shirt_number']
            except: dorsal = ''
            if 'photos' in lista_claves:
                foto = (dicc_jugador['photos']['001']['256x278'])
            else:
                foto = ''
            nombres.append(nombre)
            apodos.append(apodo)
            fechas_nacimiento.append(fecha_nacimiento)
            pesos.append(peso)
            alturas.append(altura)
            paises.append(pais)
            posiciones.append(posicion)
            equipos.append(equipo)
            dorsales.append(dorsal)
            fotos.append(foto)  # Final del bucle FOR
    columnas = ['jugador','apodo','fecha','peso','altura','pais','posicion','equipo','dorsal','url']
    df_jugadores = pd.DataFrame([nombres, apodos, fechas_nacimiento, pesos, alturas, paises, 
                                posiciones, equipos, dorsales, fotos], index=columnas)
    return df_jugadores.T # Se devuelve el dataframe transpuesto

In [234]:
df_jugadores = obtenerDatosJugadores()

In [235]:
df_jugadores

,jugador,apodo,fecha,peso,altura,pais,posicion,equipo,dorsal,url
0,Julen Agirrezabala,Agirrezabala,2000-12-26T00:00:00+00:00,80,187,ES,Portero,Athletic Club,13,https://assets.laliga.com/squad/2024/t174/p470...
1,Unai Simón,Unai Simon,1997-06-11T00:00:00+00:00,88,190,ES,Portero,Athletic Club,1,https://assets.laliga.com/squad/2024/t174/p212...
2,Dani Vivian,Vivian,1999-07-05T00:00:00+00:00,78,183,ES,Defensa,Athletic Club,3,https://assets.laliga.com/squad/2024/t174/p437...
3,Íñigo Lekue,I. Lekue,1993-05-04T00:00:00+00:00,70,180,ES,Defensa,Athletic Club,15,https://assets.laliga.com/squad/2024/t174/p197...
4,Imanol García de Albéniz,Imanol,2000-06-08T00:00:00+00:00,63,177,ES,Defensa,Athletic Club,,https://assets.laliga.com/squad/2024/t174/p470...
...,...,...,...,...,...,...,...,...,...,...
326,Santi Comesaña,Santi C.v.,1996-10-05T00:00:00+00:00,75,184,ES,Centrocampista,Villarreal Club de Fútbol SAD,4,https://assets.laliga.com/squad/2024/t449/p230...
327,Gerard Moreno,Gerard,1992-04-07T00:00:00+00:00,77,180,ES,Delantero,Villarreal Club de Fútbol SAD,7,https://assets.laliga.com/squad/2024/t449/p937...
328,Alfonso Pedraza,A. Pedraza,1996-04-09T00:00:00+00:00,73,184,ES,Delantero,Villarreal Club de Fútbol SAD,24,https://assets.laliga.com/squad/2024/t449/p208...
329,Yeremy Pino,Yeremy,2002-10-20T00:00:00+00:00,65,172,ES,Delantero,Villarreal Club de Fútbol SAD,21,https://assets.laliga.com/squad/2024/t449/p488...


In [242]:
# Tratamiento de los datos
df_jugadores['fecha'] = df_jugadores.fecha.str.slice(start=0, stop=10)
df_jugadores

,jugador,apodo,fecha,peso,altura,pais,posicion,equipo,dorsal,url
0,Julen Agirrezabala,Agirrezabala,2000-12-26,80,187,ES,Portero,Athletic Club,13,https://assets.laliga.com/squad/2024/t174/p470...
1,Unai Simón,Unai Simon,1997-06-11,88,190,ES,Portero,Athletic Club,1,https://assets.laliga.com/squad/2024/t174/p212...
2,Dani Vivian,Vivian,1999-07-05,78,183,ES,Defensa,Athletic Club,3,https://assets.laliga.com/squad/2024/t174/p437...
3,Íñigo Lekue,I. Lekue,1993-05-04,70,180,ES,Defensa,Athletic Club,15,https://assets.laliga.com/squad/2024/t174/p197...
4,Imanol García de Albéniz,Imanol,2000-06-08,63,177,ES,Defensa,Athletic Club,,https://assets.laliga.com/squad/2024/t174/p470...
...,...,...,...,...,...,...,...,...,...,...
326,Santi Comesaña,Santi C.v.,1996-10-05,75,184,ES,Centrocampista,Villarreal Club de Fútbol SAD,4,https://assets.laliga.com/squad/2024/t449/p230...
327,Gerard Moreno,Gerard,1992-04-07,77,180,ES,Delantero,Villarreal Club de Fútbol SAD,7,https://assets.laliga.com/squad/2024/t449/p937...
328,Alfonso Pedraza,A. Pedraza,1996-04-09,73,184,ES,Delantero,Villarreal Club de Fútbol SAD,24,https://assets.laliga.com/squad/2024/t449/p208...
329,Yeremy Pino,Yeremy,2002-10-20,65,172,ES,Delantero,Villarreal Club de Fútbol SAD,21,https://assets.laliga.com/squad/2024/t449/p488...


In [243]:
df_jugadores.to_excel('info_jugadores_la_liga.xlsx', index=False)   # Guardar los datos en un excel